In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from tqdm import tqdm
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_labels

# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 홀 채우기 함수 수정
def mask_filling(mask):
    # 마스크 내부 영역 추출
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_mask = np.zeros_like(mask)
    cv2.drawContours(filled_mask, contours, -1, 255, -1)
    return filled_mask

# 라마크 알고리즘을 사용하여 경계를 직선으로 근사화하는 함수
def line_approximation(contour):
    approx_curve = cv2.approxPolyDP(contour, epsilon=0.01*cv2.arcLength(contour, True), closed=True)
    return approx_curve


# CSV 파일 로드
csv_file = pd.read_csv('../submit/b5_jhsstride_040.csv')  # CSV 파일 경로

# 결과를 저장할 새로운 DataFrame 생성
result_data = pd.DataFrame(columns=['img_id', 'mask_rle'])

# 각 사진과 마스크에 대해 예시 출력 및 결과 저장
for i in tqdm(range(60640)):
    # 사진 파일 경로 생성
    
    image_path = f'../data/test_img/{csv_file["img_id"][i]}.png'  # 이미지 파일 경로 생성 (확장자에 따라 수정해야 할 수 있습니다)

    # 사진 파일 확인
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found: {image_path}")
    except Exception as e:
        print(str(e))
        assert False

    # 마스크 정보 로드 및 디코딩
    mask_data = csv_file['mask_rle'][i]
    mask = rle_decode(mask_data, image.shape[:2])

    # 작은 객체 제거
    min_object_size = 10  # 임의의 작은 객체 크기 기준 설정
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_object_size:
            cv2.drawContours(mask, [contour], 0, 0, -1)

    # 마스크 내부 영역 채우기
    filled_mask = mask_filling(mask)

    # 직선 근사화된 경계 생성
    approx_contours = [line_approximation(contour) for contour in contours]
    line_approximation_mask = np.zeros_like(filled_mask)
    cv2.drawContours(line_approximation_mask, approx_contours, -1, 255, 1)

    # 경계 내부 영역 채우기
    filled_line_approximation_mask = mask_filling(line_approximation_mask)

    # DenseCRF2D 입력 설정

    ############## 재욱이 추가 부분 ##################
    # unary = np.expand_dims(mask, axis=0)
    # unary = np.concatenate([unary, 1-unary], axis=0)
    # unary = unary.reshape((2, -1))
    # unary_float = unary.astype(np.float32)

    # # DenseCRF2D 객체 생성
    # d = dcrf.DenseCRF2D(224, 224, 2)  # width, height, nlabels

    # # unary potential 설정
    # d.setUnaryEnergy(unary_float)

    # d.addPairwiseGaussian(sxy=3, compat=3)

    # # DenseCRF2D 수행
    # Q = d.inference(10)  # 반복 횟수 설정

    # # 최적화된 마스크 가져오기
    # filled_mask = np.argmax(Q, axis=0).reshape((224, 224))


    # 결과를 DataFrame에 저장
    result_data.loc[i] = [csv_file['img_id'][i], rle_encode(filled_line_approximation_mask)]

# 결과 DataFrame을 final.csv 파일로 저장
result_data['mask_rle'] = result_data['mask_rle'].fillna(-1)
print(result_data.head)
result_data.to_csv('../submit/b5_jhsstride_040_yjypost_new.csv', index=False)

100%|██████████| 60640/60640 [29:40<00:00, 34.05it/s]  


<bound method NDFrame.head of            img_id                                           mask_rle
0      TEST_00000  20002 9 20226 10 20258 2 20450 11 20480 6 2067...
1      TEST_00001  35838 3 36059 6 36282 7 36505 8 36728 9 36950 ...
2      TEST_00002                                                   
3      TEST_00003  21 25 73 12 244 27 297 12 467 28 521 11 690 30...
4      TEST_00004  16890 23 17114 46 17338 46 17562 46 17785 47 1...
...           ...                                                ...
60635  TEST_60635                                                   
60636  TEST_60636  31542 4 31765 6 31989 6 32213 6 32438 5 32662 ...
60637  TEST_60637  57 22 88 19 281 22 312 19 504 23 536 19 728 23...
60638  TEST_60638  14486 5 14702 15 14922 20 15145 21 15368 22 15...
60639  TEST_60639  11011 3 11230 9 11448 15 11667 21 11888 24 121...

[60640 rows x 2 columns]>


In [2]:
result_data = pd.read_csv('../submit/b5_jhsstride_040_yjypost_new.csv')  # CSV 파일 경로
result_data['mask_rle'] = result_data['mask_rle'].fillna(-1)
print(result_data.head)
result_data.to_csv('../submit/b5_jhsstride_040_yjypost_new.csv', index=False)

<bound method NDFrame.head of            img_id                                           mask_rle
0      TEST_00000  20002 9 20226 10 20258 2 20450 11 20480 6 2067...
1      TEST_00001  35838 3 36059 6 36282 7 36505 8 36728 9 36950 ...
2      TEST_00002                                                 -1
3      TEST_00003  21 25 73 12 244 27 297 12 467 28 521 11 690 30...
4      TEST_00004  16890 23 17114 46 17338 46 17562 46 17785 47 1...
...           ...                                                ...
60635  TEST_60635                                                 -1
60636  TEST_60636  31542 4 31765 6 31989 6 32213 6 32438 5 32662 ...
60637  TEST_60637  57 22 88 19 281 22 312 19 504 23 536 19 728 23...
60638  TEST_60638  14486 5 14702 15 14922 20 15145 21 15368 22 15...
60639  TEST_60639  11011 3 11230 9 11448 15 11667 21 11888 24 121...

[60640 rows x 2 columns]>
